## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Data Exploration

**Challenge**: Read the .csv file and store it in a Pandas dataframe

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
df = pd.read_csv('QueryResults.csv', names=['DATE', 'TAG', 'POSTS'], header=0)

**Challenge**: Examine the first 5 rows and the last 5 rows of the of the dataframe

In [ ]:
df

**Challenge:** Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [ ]:
df.shape  # To show (rows,columns)

**Challenge**: Count the number of entries in each column of the dataframe

In [ ]:
df.count()

**Challenge**: Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [ ]:
languages = df.iloc[:, 1].unique()
num_post_lang = {}
for language in languages:
    language_n = df[df.iloc[:, 1] == language]
    sum_value = int(language_n.iloc[:, 2].sum())
    num_post_lang[language] = sum_value
lan_df = pd.DataFrame.from_dict(num_post_lang, orient='index', columns=['POSTS'])
most_used_lan_df = lan_df.sort_values(by='POSTS', ascending=False)
most_used_lan_df

In [ ]:
df.iloc[:, [1,2]].groupby('TAG').sum().sort_values(by='POSTS', ascending=False)

Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

**Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


In [ ]:
df.groupby('TAG').count()

## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [ ]:
df.iloc[:, 0] = pd.to_datetime(df.iloc[:,0]).dt.date

In [ ]:
# df['DATE'] = pd.to_datetime(df['DATE'])

In [ ]:
type(df.iloc[0,0])

In [ ]:
df.head(15)

## Data Manipulation



In [ ]:
data = {
    'Age': ['Young', 'Young', 'Young', 'Young', 'Old', 'Old', 'Old', 'Old'],
    'Actor': ['Jack', 'Arnold', 'Keanu', 'Sylvester', 'Jack', 'Arnold', 'Keanu', 'Sylvester'],
    'Power': [100, 80, 25, 50, 99, 75, 5, 30]
       }
test_df = pd.DataFrame(data=data)
test_df.pivot(values='Power', columns='Actor', index='Age')

In [ ]:
reshape_df = df.pivot(values='POSTS', index='DATE', columns='TAG')

**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

In [ ]:
reshape_df.head(5)

In [ ]:
reshape_df.shape

In [ ]:
reshape_df.count()

In [ ]:
reshape_df.fillna(0, inplace=True)

In [ ]:
# reshape_df = reshape_df.fillna(0)

In [ ]:
reshape_df.head(3)

**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

In [ ]:
reshape_df.count()

In [ ]:
reshape_df.sum()

In [ ]:
reshape_df.isna().values.any()

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart.

In [ ]:
plt.figure(figsize=(16,3))
plt.ylabel('Posts', fontsize=14)
plt.xlabel('date', fontsize=14)
plt.ylim(0, 35000)
plt.plot(reshape_df.index, reshape_df.javascript)

In [ ]:
# plt.plot(reshape_df.index, reshape_df['python'])

In [ ]:
# plt.plot(reshape_df.index, reshape_df.iloc[:,10])

In [ ]:
# plt.plot(reshape_df.index, reshape_df.python, 'blue',  reshape_df.javascript, 'orange')

**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

In [ ]:
# print(reshape_df.columns)
# plt.plot(reshape_df.index, reshape_df.iloc[:, 6], 'orange' )
# plt.plot(reshape_df.index, reshape_df.[:, 10], 'blue')

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 

In [ ]:
# The window is number of observations that are averaged
reshape_df_window = reshape_df.rolling(window=4).mean()

plt.figure(figsize=(15, 10))
plt.xlabel('Date', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(0, 30000)
plt.ylabel('Number of Posts', fontsize=14)
plt.plot(reshape_df_window.index, reshape_df_window.iloc[:, :],
         label=reshape_df.columns, linewidth=3,)
plt.suptitle("Stack Overflow's Most Popular Languages",fontsize=16)
plt.legend(loc='best', fontsize=17)
